In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
import catboost as cb
sb.set() 

In [17]:
# Popularity and profit statistics
success_data = pd.read_pickle('success.pkl')
popularity_data = success_data[['popularity']]
profit_data = success_data[['profit']]
factors = pd.read_pickle('factors.pkl')
director = pd.read_pickle('director.pkl')
genre = pd.read_pickle('encoded_genre.pkl')
cast = pd.read_pickle('cast.pkl')
prod_comp = pd.read_pickle('production_companies.pkl')
release_year = pd.DataFrame(factors['release_year'])
release_month = pd.DataFrame(factors['month'])
budget = pd.DataFrame(factors['budget'])
runtime = pd.DataFrame(factors['runtime'])

In [18]:
data = pd.concat([success_data, director, genre, cast, prod_comp, factors], axis=1)
data

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [13]:
# create appropriate datasets for Train and Test in an 80:20 ratio 
# Train Set : 80%
from sklearn.model_selection import train_test_split
factors_train, factors_test = train_test_split(factors, test_size=0.2, random_state=0)
success_train, success_test = train_test_split(success_data, test_size=0.2, random_state=0)

print("Train Set :", factors_train.shape, success_train.shape)
print("Test Set  :", factors_test.shape, success_train.shape)


Train Set : (1029, 9) (1029, 2)
Test Set  : (258, 9) (1029, 2)


We utilised Cat Boost to extract the all the factors we felt could affect the popularity and profit, which will indicate the success rate of each movie.

In [14]:
train_dataset = cb.Pool(factors_train, success_train['popularity']) 
test_dataset = cb.Pool(factors_test, success_test['popularity'])

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="Owen Wilson|Jennifer Aniston|Eric Dane|Kathleen Turner|Alan Arkin": Cannot convert 'b'Owen Wilson|Jennifer Aniston|Eric Dane|Kathleen Turner|Alan Arkin'' to float

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE')

grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)